main.ipynbで既にモデルを作成した場合、こちらのファイルを使用する。

#---------------------------------------------------------------------------------------------------#

<h2>初期設定</h2>

<h3>1.各種ライブラリのインポート</h3>

In [1]:
import sys
import os
import gc
import re
import copy
import math
import tqdm
import itertools
import shutil
import pickle
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
from scipy import signal
%matplotlib inline

<h3>2.whale_call_packageディレクトリを格納している絶対パスを記述.</h3>
windows,macでそれぞれ用意しているが不要な方はコメントアウト

In [2]:
#windows
sys.path.append("C:/Users/")

#mac
sys.path.append("/Users/nakahigashikazuo3/Desktop")

<h3>3.whale_call_packageをインポート</h3>

In [3]:
from whale_call_package import whale_call as wh
from whale_call_package import parameter_setting as para

In [4]:
#OBSナンバーを選択。数字6桁以上は避けてほしい。
obs = ''


<h3>4.データを出力するディレクトリ、データが格納されているディレクトリを記述。</h3>不要な方をコメントアウト

datadirectoryの方をobsデータの入っている絶対パスに変更するだけでOK。

In [5]:
#windows
workdir_base = f'D:/whale/data/WhaleCallField'
preworkdirectory=f'{workdir_base}/{obs}'
datadirectory=f'D:/whale/data/origindata/{obs}'
#datadirectory=f'D:/whale/JB17a/{obs}'

#mac
preworkdirectory=f'/Volumes/LaCie/WhaleCallField/{obs}'
datadirectory=f'/Volumes/LaCie/JS18a/data/{obs}'

<h3>5.パラメータ変更用に作業ディレクトリ下にディレクトリを設定する</h3>

In [6]:
currentdirectory = '5' #任意の名前をつける.数字４文字以上は不具合が出る
parameter_number = '2' #whale_call_package内のparameter_setting.pyを確認

In [7]:
#環境設定をインスタンス化し、作業ディレクトリを作成、waveファイルのリストを取得する
workdirectory=preworkdirectory+'/'+currentdirectory
whale = wh.SetField(workdirectory=workdirectory, datadirectory=datadirectory)
whale.make_directory()
filelist = whale.get_wavfilelist()

<h3>6.処理を行うwavファイルを指定する場合以下を実行する。</h3>不要ならば全てコメントアウト

<h4>1．任意で指定する。</h4>

In [8]:
#filelist = whale.pick_wavfilelist(filelist=filelist, month=["1904"])
#"1808", "1809", "1810", "1811", "1812", "1901", , "1903", "1904","1905","1906","1907"

<h4>2．ランダムで指定する。</h4>

In [9]:
#filelist = whale.random_pick_wavfilelist(filelist=filelist, pick_num=5)

<h4>3.不要なファイルを指定して削除する。</h4>

In [10]:
#filelist = whale.del_wavfile(filelist=filelist, del_list=['180717','180718'])

<h4>4.最初の２日を排除する。</h4>

In [11]:
filelist = filelist[2:]

取得したファイル一覧

In [12]:
filelist

['D:/whale/data/origindata/1805_2020\\180727-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180728-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180729-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180730-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180731-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180801-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180802-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180803-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180804-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180805-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180806-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180807-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180808-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180809-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180810-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180811-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180812-000000.wav',
 'D:/whale/dat

In [13]:
#各パラメータセッティング
sr, n_fft, hop_length, fmin, fmax, fpass, fstop, gpass, gstop, num_train, num_guard, rate_fa = para.para(parameter_number)

In [14]:
#各パラメータのログ出力
para.make_parameter_log(workdirectory=workdirectory, sr=sr, n_fft=n_fft, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

In [15]:
#グラフ描画用のスケール取得
day_scale, count_scale = whale.timescales(filelist=filelist)

<h3>7.各処理の実行有無を決定</h3>

In [38]:
#Trueで実行する

#cfarの実行
cfar_flag=True
#スペクトログラム画像の生成
ms_flag=False
#鳴音を基に各種グラフの作成
call_graph_flag = False
#モデルを使用して画像を分類する
sep_fig_flag = False
#n_callの分類
sep_n_call = False
#分類後画像の情報を保存する
save_calls = False
#中心周波数を抽出
centroid_flag = False
#IPIを抽出
ipi_flag = False

#---------------------------------------------------------------------------------------------------#

<h2>CFARに関する処理</h2>

<h3>1.初期設定</h3>

In [16]:
cfar = wh.Cfar(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

パラメータ
・parameter_setting.pyに記載されている各パラメータ
・保存するsplit音声ファイルの長さ

#---------------------------------------------------------------------------------------------------#

<h3>3.+cfar処理</h3>

cfar処理を実行し、検出した部分を中心にn秒に分割したwaveファイルを作成する.nを変更する場合はtimeを変更.n秒データを出力しない場合にはmake_split_wavefileをコメントアウトする

中断可能

In [34]:
temp_file = filelist

In [35]:
#中断した場合、スタート位置をずらす
#temp_file = filelist[342:]

In [36]:
temp_file

['D:/whale/data/origindata/1805_2020\\180727-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180728-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180729-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180730-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180731-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180801-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180802-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180803-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180804-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180805-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180806-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180807-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180808-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180809-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180810-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180811-000000.wav',
 'D:/whale/data/origindata/1805_2020\\180812-000000.wav',
 'D:/whale/dat

In [39]:
if cfar_flag:
    for i in temp_file:
        name = cfar.make_cfar_dir(i)
        y, srt = cfar.get_wav(i)
        data = cfar.bandpass(x=y, freqpass=fpass, freqstop=fstop, gpass=gpass, gstop=gstop)
        peak_idx = cfar.run_cfar(data=data, name=name) #CFARを実行し、検出した時間と個数を各日付で返し、指定したファイルのリストを出力。
        cfar.make_split_wavfile(time=3, peak_idx=peak_idx, name=name, data=y) #time=n秒のwavファイルを出力

KeyboardInterrupt: 

#---------------------------------------------------------------------------------------------------#

<h2>スペクトログラム作成</h2>

<h3>1.初期設定</h3>

In [ ]:
spec = wh.MakeSpectrogram(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

#---------------------------------------------------------------------------------------------------#

<h3>3.+n秒画像作成</h3>

中断可能

In [ ]:
if ms_flag:
    picked = spec.get_sp_wav(filelist=filelist)
    temp_picked = picked

In [ ]:
#中断した場合、スタート位置をずらす
#temp_picked=picked[116:]

In [ ]:
if ms_flag:
    print(temp_picked)

In [ ]:
if ms_flag:
    def_figsize = plt.rcParams.get('figure.figsize')
    def_dpi = plt.rcParams.get('figure.dpi') 
    os.makedirs(f'{workdirectory}/cfar/all/fig_convert', exist_ok=True)
    for i in temp_picked:
        spec.make_sp_spec_all(day=i)
        gc.collect()
    plt.rcParams['figure.figsize'] = def_figsize
    plt.rcParams['figure.dpi'] = def_dpi

#---------------------------------------------------------------------------------------------------#

<h2>Callをグラフで表示</h2>

全て実行するとワークディレクトリに各種グラフが生成される。

In [ ]:
%matplotlib inline
#読み込むpeaktextの種類と保存するディレクトリ名を決定する
peak_file = 'peak.txt'
dir_name = 'day_count'

In [ ]:
if call_graph_flag:
    cfar.make_call_graph(filelist=filelist,peak_file=peak_file, dir_name=dir_name, day_scale=day_scale, count_scale=count_scale)

#---------------------------------------------------------------------------------------------------#

<h2>作成したモデルを基に実際に仕分けをする</h2>

In [ ]:
cnn = wh.Cnn(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

In [ ]:
cnn.model_cp(model_dir=f'D:/whale/data/WhaleCallField/1805_2020/5/cfar/all/model')

In [ ]:
dir_list = ['sentering', 'noise', 'call', 'n_call']
sep_img_list =  f'{workdirectory}/cfar/all/fig_convert'
model_name = 'my_model.h5'
days= os.listdir(f'{workdirectory}/cfar')
days.remove('all')
days.remove('pick_list.txt')

In [ ]:
all_result = [0]*4
if sep_fig_flag:
    for i in days:
        all_result = cnn.sep_cnn_noise(test_img_list=sep_img_list,detected_dir=f'{workdirectory}/cfar/all/sep_fig', model_name=model_name, days=i, all_result=all_result)

In [ ]:
all_result

In [ ]:
if sep_fig_flag:
    cnn.noise_graph(filelist=filelist,new_or='before', day_scale=day_scale, count_scale=count_scale)

#---------------------------------------------------------------------------------------------------#

<h2>n_call内の鳴音を再分類</h2>

In [ ]:
ncall = wh.NCallAnalysis(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

In [ ]:
if sep_n_call:
    test_img_list = f'{workdirectory}/cfar/all/n_call_fig'
    detected_dir = f'{workdirectory}/cfar/all/sep_n_call'
    model_name = 'my_model.h5'
    days= os.listdir(f'{workdirectory}/cfar')
    days.remove('all')
    days.remove('pick_list.txt')
    os.makedirs(detected_dir,exist_ok=True)

In [ ]:
if sep_n_call:
    ncall.up_ncall(n_call_list_dir='cfar/all/sep_fig/n_call', mode='volumeup')
    all_result = [0]*4
    for i in days:
        all_result = cnn.sep_cnn_noise(test_img_list=test_img_list, detected_dir=detected_dir, model_name=model_name, days=i, all_result=all_result)
    print('all_result:',all_result)

In [ ]:
if sep_n_call:
    ncall.up_ncall(n_call_list_dir='cfar/all/sep_n_call/n_call', mode='plus_noise')
    ncall.del_ncall(path=f'{workdirectory}/cfar/all/sep_n_call/n_call', check=True)
    all_result = [0]*4
    for i in days:
        all_result = cnn.sep_cnn_noise(test_img_list=test_img_list, detected_dir=detected_dir, model_name=model_name, days=i, all_result=all_result)
    print('all_result:',all_result)
    ncall.move_ncall(data_dir=f'{workdirectory}/cfar/all/sep_n_call', target_dir=f'{workdirectory}/cfar/all/sep_fig')

#---------------------------------------------------------------------------------------------------#

<h2>分類したデータをテキストファイルで保存し、グラフを出力する</h2>

In [ ]:
if save_calls:
    cnn.make_peaktxt(filelist=filelist, dir_list=dir_list)

In [ ]:
if save_calls:
    dirs = 'call'
    os.makedirs(f'{workdirectory}/fig', exist_ok=True)
    cfar.make_call_graph(filelist=filelist,peak_file=f'peak_{dirs}.txt', dir_name=f'day_count_{dirs}', day_scale=day_scale, count_scale=count_scale)

In [ ]:
if save_calls:
    cnn.noise_graph(filelist=filelist,new_or='new', day_scale=day_scale, count_scale=count_scale)

#---------------------------------------------------------------------------------------------------#

<h2>画像から中心周波数等のパラメータを抽出する</h2>

<h3>1.初期設定</h3>

In [ ]:
analysis = wh.Analysis(workdirectory=workdirectory, datadirectory=datadirectory, sr=sr, n_fft=n_fft, hop_length=hop_length, fmin=fmin, fmax=fmax, fpass=fpass, fstop=fstop, gpass=gpass, gstop=gstop, num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

<h3>中心周波数を抽出</h3>

中心周波数を求めたリストを作成

In [ ]:
if centroid_flag:
    centroid_list = analysis.run_centroid(filelist=filelist)

In [ ]:
if 'centroid_list.txt' not in os.listdir(f'{workdirectory}/analysis'):
    with open(f'{workdirectory}/analysis/centroid_list.txt', 'wb') as f:
        pickle.dump(centroid_list, f)
if 'centroid_list.txt' in os.listdir(f'{workdirectory}/analysis'):
    if 'temp_centroid_list.txt' in os.listdir(f'{workdirectory}/analysis'):
        os.remove(f'{workdirectory}/analysis/temp_centroid_list.txt')

In [ ]:
with open(f'{workdirectory}/analysis/centroid_list.txt', 'rb') as f:
    list_row = pickle.load(f)

In [ ]:
if centroid_flag:
    centroid_list = analysis.temp_cent(filelist=filelist, list_row=list_row)

call数と移動平均線のグラフを作成

In [ ]:
analysis.callma_graph(list_row=list_row, count_scale=count_scale, day_scale=day_scale)

扱いやすいように変換

In [ ]:
all_cent_list = []
all_time_list = []
for j in range(len(list_row)):
    cent_list = [list_row[j][i][1] for i in range(len(list_row[j]))]
    time_list = [list_row[j][i][0] for i in range(len(list_row[j]))]
    all_cent_list = all_cent_list + cent_list
    all_time_list = all_time_list + time_list

IPIを求めてリストを作成

In [ ]:
if ipi_flag:
    ipi_list, ipi_all_list = analysis.ipi(all_time_list=all_time_list, limsec=45)

In [ ]:
os.makedirs(f'{workdirectory}/analysis', exist_ok=True)
if 'ipi_list.txt' not in os.listdir(f'{workdirectory}/analysis'):
    with open(f'{workdirectory}/analysis/ipi_list.txt', 'wb') as f:
        pickle.dump(ipi_list, f)
if 'ipi_all_list.txt' not in os.listdir(f'{workdirectory}/analysis'):
    with open(f'{workdirectory}/analysis/ipi_all_list.txt', 'wb') as f:
        pickle.dump(ipi_all_list, f)

In [ ]:
with open(f'{workdirectory}/analysis/ipi_list.txt', 'rb') as f:
    ipi_row = pickle.load(f)
with open(f'{workdirectory}/analysis/ipi_all_list.txt', 'rb') as f:
    ipi_all_row = pickle.load(f)

IPIのヒストグラムを作成

In [ ]:
analysis.ipi_histgram(ipi_row=ipi_row, bins=300)

年間の1日あたりの平均IPIと移動平均線のグラフを作成

In [ ]:
ipi_average = analysis.ipima_graph(ipi_row=ipi_row, count_scale=count_scale, day_scale=day_scale)

中心周波数のヒストグラムを作成

In [ ]:
analysis.cent_histgram(all_cent_list=all_cent_list, bins=200)

年間の１日辺りの中心周波数と移動平均線のグラフを作成

In [ ]:
day_cent_list = analysis.centroid_timescales_graph(count_scale=count_scale,day_scale=day_scale)

call数、中心周波数、IPIを同一スケールに変換、相関関係を見るグラフを作成

In [ ]:
day_list = [len(list_row[i]) for i in range(len(list_row))]
analysis.ma_graph(day_list=day_list, ipi_average=ipi_average, day_cent_list=day_cent_list,count_scale=count_scale,day_scale=day_scale)

１日ごとの中心周波数

In [ ]:
analysis.cent_day_graph(filelist=filelist)

1日毎の休符間隔

In [ ]:
analysis.ipi_day_graph(filelist=filelist, ipi_all_row=ipi_all_row)

1日毎の中心周波数と休符間隔

In [ ]:
analysis.ipi_cent_graph(filelist=filelist)

signale_ipiで指定した範囲内の鳴音を抽出、グラフを作成

In [ ]:
analysis.detect_singlecall(ipi_all_row=ipi_all_row, single_ipi=[30,40], count_scale=count_scale,day_scale=day_scale)

#---------------------------------------------------------------------------------------------------#

ipiのシーケンス分析

45秒以下の休符間隔で5個以上連続する鳴音をシーケンスとして抽出
１つ目の鳴音の中心周波数を省き、休符間隔と個数を合わせている

In [ ]:
ipi_sequence = analysis.detect_ipi_sequence(ipi_all_row=ipi_all_row, seq_min_num=5)

対象日を指定

In [ ]:
target_day = '190708'

In [ ]:
day = [day for day in filelist if target_day in day]
if len(day) == 1:
    day = day[0]
    print(f'対象日:{day}')
    day = filelist.index(day)
else:
    print('Error:対象が複数存在します。')

シーケンスを抽出

In [ ]:
peak_call_idx, peak_idx, name = analysis.get_call(filelist[day])
ipi_day_seq = []
ipi_num = []
cent_day_seq = []
for j in range(len(ipi_sequence[day])):
    a = [ipi_sequence[day][j][i][0] for i in range(len(ipi_sequence[day][j]))]
    b = [ipi_sequence[day][j][i][1]/sr for i in range(len(ipi_sequence[day][j]))]
    ab = [float(list_row[day][i][1]) for i in range(len(b))]
    ipi_num.append(a)
    ipi_day_seq.append(b)
    cent_day_seq.append(ab)

対象日のシーケンスの数

In [ ]:
len(ipi_sequence[day])

対象日の各シーケンスの鳴音数

In [ ]:
day_seq_len = [len(i) for i in ipi_sequence[day]]

In [ ]:
plt.figure()
plt.bar(range(0,len(day_seq_len),1), day_seq_len)
plt.title('sequence_length')
plt.xlabel('sequence_num')
plt.ylabel('call_num')
plt.show()

対象のシーケンス番号を指定

In [ ]:
seq = 6

In [ ]:
seq_time = [int(peak_call_idx[i][1])/sr for i in ipi_num[seq]]
seq_h = [i/3600 for i in seq_time]
seq_s = round(seq_h[0],2)
seq_e = round(seq_h[-1],2)
print(f'対象時間{seq_s}-{seq_e}時')

In [ ]:
ipi_day_seq[seq]

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
plt.subplots_adjust(hspace=0.6)
ax1.scatter(range(0, len(cent_day_seq[seq]),1), cent_day_seq[seq],s=15)
ax1.plot(range(0, len(cent_day_seq[seq]),1), cent_day_seq[seq])
ax1.set_xticks(np.arange(0,len(cent_day_seq[seq]) , 5))
ax1.set_title(f'{name}_{seq_s}-{seq_e}hour')
ax1.set_ylabel('centroid')
ax1.set_xlabel('sequence')

ax2.scatter(range(0, len(ipi_day_seq[seq]),1), ipi_day_seq[seq], s=15)
ax2.plot(range(0, len(ipi_day_seq[seq]),1), ipi_day_seq[seq])
ax2.set_xticks(np.arange(0,len(ipi_day_seq[seq]),5))
ax2.set_ylabel('ipi')
ax2.set_xlabel('sequence')
plt.show()
#fig.savefig(f'{workdirectory}/centroid/{name}_{seq_s}-{seq_e}hour.jpg')

#---------------------------------------------------------------------------------------------------#

OBS毎の休符間隔毎の鳴音数の割合比較

Whalecallfield直下の各OBSのディレクトリ名をリスト化する

In [ ]:
#obs_list = ['1801_2020','1802_2020','1803_2020','1804_2020','1805_2020','1806_2020','1807_2020']
obs_list = ['JS1701','JS1702','JS1703','JS1704','JS1705','JS1706','JS1707']

グラフで使用するOBS名をリスト化

In [ ]:
obs_name=['OBS1','OBS2','OBS3','OBS4','OBS5','OBS6','OBS7']

グラフのファイル名を指定

In [ ]:
filename = 'OBS17-18'

In [ ]:
ipinum = []
for i in range(len(obs_list)):
    path = f'{workdir_base}/{obs_list[i]}/{currentdirectory}'
    low, mid, high = analysis.all_obs_ipi(low_sec=5, high_sec=30, path=path)
    ipinum.append([obs_name[i], low, mid, high])
with open(f'{workdir_base}/{filename}.txt', 'w', encoding='utf-8') as f:
    for i in ipinum:
        print(*i, sep=' ', file=f)

In [ ]:
obsfile = []
with open(f'{workdir_base}/{filename}.txt', 'r', encoding='utf-8') as f:
    for line in f:
        obsfile.append(line.rstrip('\n').split(' '))

In [ ]:
obsfile

In [ ]:
for i in range(len(obsfile)):
    obsfile[i] = list(map(int, obsfile[i][1:]))

In [ ]:
obsfile

In [ ]:
df = pd.DataFrame([obsfile[0],obsfile[1],obsfile[2],obsfile[3],obsfile[4],obsfile[5],obsfile[6]],
                 index=obs_name,
                  columns=['low','mid','high'])
dataset = copy.deepcopy(df.T)

In [ ]:
dataset

In [ ]:
plot_dataset = pd.DataFrame(index = dataset.index)
for col in dataset.columns:
    plot_dataset[col] = round(100 * dataset[col] / dataset[col].sum(), 1)

In [ ]:
plot_dataset

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
for i in range(len(plot_dataset)):
    ax.bar(plot_dataset.columns, 
           plot_dataset.iloc[i], 
           bottom=plot_dataset.iloc[:i].sum()
          )
    for j in range(len(plot_dataset.columns)):
        plt.text(x=j, 
                 y=plot_dataset.iloc[:i, j].sum()+(plot_dataset.iloc[i, j]/2), 
                 s=f'{plot_dataset.iloc[i, j]}%', 
                 ha='center', 
                 va='bottom'
                )
ax.set(xlabel='OBS', ylabel='%')
ax.legend(dataset.index)
ax.set_title('OBS_IPI')
plt.savefig(f'{workdir_base}/{filename}.jpg')
plt.show()